### Under progress....

In [119]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [132]:
df = pd.read_csv("cleaned_air_bnb_Jul_26.csv")
print(df.shape)

(6136, 54)


In [133]:
df.head(2)

,url,name,numberOfGuests,address,roomType,bathroomLabel,maxNights,minNights,localizedCheckInTimeWindow,localizedCheckOutTime,city,localizedCity,state,latitude,longitude,price,hostId,memberSince,responseTime,numberOfLanguages,Washer,Shampoo,Hair dryer,Air conditioning,Private entrance,numberOfBedsAvailable,numberOfBedrooms,double_bed,floor_mattress,single_bed,queen_bed,couch,king_bed,air_mattress,sofa_bed,small_double_bed,bunk_bed,toddler_bed,crib,hammock,water_bed,allowsChildren,allowsEvents,allowsPets,allowsSmoking,allowsInfants,personCapacity,Accuracy,Check-in,Cleanliness,Communication,Location,Review Count,Value
0,https://www.airbnb.com/rooms/31573201,Avocado B&B and Cottages,4,"Ngong, Kenya",Private room in bed and breakfast,1 shared bath,1125,2,After 3:00 PM,NaN,Ngong,Ngong,NaN,-1.35691,36.66787,22.0,236673322,"January,2019",NaN,1,True,True,True,False,False,2.0,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,True,True,4,4.5,5.0,5.0,5.0,4.5,2.0,5.0
1,https://www.airbnb.com/rooms/38121290,Traditional Africa Mud House in Nairobi,2,"Nairobi, Nairobi County, Kenya",Private room in earthen home,1 shared bath,1125,1,9:00 AM - 11:00 PM,NaN,Nairobi,Nairobi,Nairobi County,-1.33553,36.67175,20.0,223132195,"October,2018",NaN,0,False,False,False,False,False,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,True,True,True,2,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [134]:
# folow data_exploration_vicky.ipynb file


In [135]:
df=df.replace({True:1,False:0,'Studio':1})
g = df.dropna(subset=['state']).drop_duplicates('city').set_index('city')['state']
df['new_state'] = df['state'].fillna(df['city'].map(g))

g = df.dropna(subset=['city']).drop_duplicates('new_state').set_index('new_state')['city']
df['new_city'] = df['city'].fillna(df['new_state'].map(g))


In [136]:
def addnewfeatures(x):
    if x.find('Private') > 0:
        #print("private")
        return 0
    elif x.find('Shared') > 0:
        #print("shared")
        return 1
    elif x.find('Entire') > 0:
        #print("Entire")
        return 2
    else:
        return 3
    
df['new_roomType'] = df.apply(lambda x: addnewfeatures(x['roomType']), axis=1)
df['bathroomLabel']=df.bathroomLabel.str.extract('(\d+)').astype("float")


In [139]:
df['amenities'] = df['Washer']+df['Shampoo']+df['Hair dryer']+df['Air conditioning']
df['bed_type'] = df['double_bed']+df['floor_mattress']+df['single_bed']+df['queen_bed']+df['couch']+df['king_bed']+df['air_mattress']+df['sofa_bed']+df['small_double_bed']+df['bunk_bed']
df.rename(columns = {'Review Count':'Review_Count'}, inplace = True)

In [140]:

df = df.drop(columns=['city','state','roomType','url','name','address','localizedCheckInTimeWindow','localizedCheckOutTime',
                      'responseTime','localizedCity','hostId','memberSince','numberOfLanguages','numberOfGuests',
                      'maxNights','minNights','latitude','longitude','Washer','Shampoo','Hair dryer','Air conditioning','double_bed','floor_mattress','single_bed','queen_bed','couch','king_bed','air_mattress','sofa_bed','small_double_bed','bunk_bed'],axis=1)
df = df.dropna(subset = ['price','bathroomLabel'])
indexPrice = df[ df['price'] >= 200 ].index
df.drop(indexPrice, inplace=True)

df['bathroomLabel'].astype("float")
df['numberOfBedrooms'].astype("float")

In [141]:
df.head(2)

,bathroomLabel,price,Private entrance,numberOfBedsAvailable,numberOfBedrooms,toddler_bed,crib,hammock,water_bed,allowsChildren,allowsEvents,allowsPets,allowsSmoking,allowsInfants,personCapacity,Accuracy,Check-in,Cleanliness,Communication,Location,Review_Count,Value,new_state,new_city,new_roomType,amenities,bed_type
0,1.0,22.0,0.0,2.0,2,0.0,0.0,0.0,0.0,0,0,0,1,1,4,4.5,5.0,5.0,5.0,4.5,2.0,5.0,Wilaya ya Kajiado,Ngong,3,3,2.0
1,1.0,20.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,1,0,1,1,1,2,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Nairobi County,Nairobi,3,0,0.0


In [142]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3539 entries, 0 to 6135
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   bathroomLabel          3539 non-null   object 
 1   price                  3539 non-null   float64
 2   Private entrance       3539 non-null   float64
 3   numberOfBedsAvailable  3539 non-null   float64
 4   numberOfBedrooms       3539 non-null   object 
 5   toddler_bed            3539 non-null   float64
 6   crib                   3539 non-null   float64
 7   hammock                3539 non-null   float64
 8   water_bed              3539 non-null   float64
 9   allowsChildren         3539 non-null   int64  
 10  allowsEvents           3539 non-null   int64  
 11  allowsPets             3539 non-null   int64  
 12  allowsSmoking          3539 non-null   int64  
 13  allowsInfants          3539 non-null   int64  
 14  personCapacity         3539 non-null   int64  
 15  Accu

0       1.0
1       1.0
2       1.0
3       3.0
4       3.0
       ... 
6131    1.0
6132    1.0
6133    1.0
6134    1.0
6135    1.0
Name: bathroomLabel, Length: 5521, dtype: float64

In [130]:
df.dropna()

,bathroomLabel,price,Private entrance,numberOfBedsAvailable,numberOfBedrooms,toddler_bed,crib,hammock,water_bed,allowsChildren,allowsEvents,allowsPets,allowsSmoking,allowsInfants,personCapacity,Accuracy,Check-in,Cleanliness,Communication,Location,Review_Count,Value,new_state,new_city,new_roomType,amenities,bed_type
0,1,22.0,0.0,2.0,2,0.0,0.0,0.0,0.0,0,0,0,1,1,4,4.5,5.0,5.0,5.0,4.5,2.0,5.0,Wilaya ya Kajiado,Ngong,3,3,2.0
7,1,20.0,0.0,1.0,1,0.0,0.0,0.0,0.0,1,1,1,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Nairobi County,Nairobi,3,1,1.0
9,1,24.0,0.0,0.0,1,0.0,0.0,0.0,0.0,1,0,0,1,1,2,5.0,5.0,5.0,4.9,4.8,9.0,4.8,Nairobi County,Nairobi,3,1,0.0
12,1,20.0,0.0,2.0,1,0.0,0.0,0.0,0.0,0,0,1,0,0,2,5.0,4.0,4.0,5.0,3.0,1.0,4.0,Nairobi County,Nairobi,3,2,0.0
17,1,18.0,1.0,1.0,1,0.0,0.0,0.0,0.0,1,0,0,0,1,2,4.8,4.8,5.0,4.6,4.8,5.0,5.0,Nairobi County,Nairobi,3,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6124,1,17.0,1.0,2.0,1,0.0,0.0,0.0,0.0,1,0,0,1,0,3,4.5,5.0,5.0,4.0,4.0,2.0,5.0,Nairobi County,Nairobi,3,1,2.0
6126,1,17.0,0.0,1.0,1,0.0,0.0,0.0,0.0,1,0,0,0,0,1,5.0,5.0,5.0,5.0,5.0,1.0,5.0,Nairobi County,Nairobi,3,1,1.0
6130,1,22.0,1.0,2.0,2,0.0,0.0,0.0,0.0,1,1,1,0,1,2,5.0,5.0,5.0,5.0,5.0,1.0,5.0,Trans-Nzoia County,Kitale,3,3,2.0
6134,1,20.0,0.0,2.0,2,0.0,0.0,0.0,0.0,1,1,1,0,1,4,5.0,5.0,5.0,5.0,5.0,1.0,5.0,Trans-Nzoia County,Kitale,3,0,2.0
